<a href="https://colab.research.google.com/github/shreyasgowdac-319/1BM23CS319-BIS-LAB/blob/main/labsee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import multiprocessing as mp
import random
import time

# =========================
# USER INPUT
# =========================
def get_user_input():
    print("=== Parallel Urban Cellular Automata Simulation ===")
    rows = int(input("Enter number of rows (e.g. 300): "))
    cols = int(input("Enter number of columns (e.g. 300): "))
    steps = int(input("Enter number of simulation steps (e.g. 20): "))
    processors = int(input("Enter number of processors (e.g. 4): "))
    threshold = float(input("Enter urbanization threshold (0–1, e.g. 0.6): "))
    return rows, cols, steps, processors, threshold


# =========================
# GRID INITIALIZATION
# =========================
def initialize_grid(rows, cols):
    grid = np.zeros((rows, cols), dtype=np.uint8)

    # Randomly seed urban cells (initial development)
    seed_count = (rows * cols) // 25
    for _ in range(seed_count):
        x = random.randint(0, rows - 1)
        y = random.randint(0, cols - 1)
        grid[x, y] = 1

    return grid


# =========================
# NEIGHBORHOOD DENSITY
# =========================
def neighborhood_density(grid, i, j):
    rows, cols = grid.shape
    urban = 0
    total = 0

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            ni, nj = i + dx, j + dy
            if 0 <= ni < rows and 0 <= nj < cols:
                urban += grid[ni, nj]
                total += 1

    return urban / total if total > 0 else 0


# =========================
# LINE-SCANNING LOAD BALANCE
# =========================
def line_scanning(rows, processors):
    base = rows // processors
    ranges = []
    start = 0

    for p in range(processors):
        end = start + base
        if p == processors - 1:
            end = rows
        ranges.append((start, end))
        start = end

    return ranges


# =========================
# UPDATE BLOCK (PARALLEL CA)
# =========================
def update_block(args):
    grid, start_row, end_row, threshold = args
    rows, cols = grid.shape
    new_block = grid[start_row:end_row].copy()

    for i in range(start_row, end_row):
        for j in range(cols):
            if grid[i, j] == 0:
                density = neighborhood_density(grid, i, j)
                random_pressure = random.random()

                # CA transition rule
                if density + random_pressure > threshold:
                    new_block[i - start_row, j] = 1

    return new_block


# =========================
# PARALLEL URBAN CA SIMULATION
# =========================
def parallel_urban_CA(grid, steps, processors, threshold):
    rows, cols = grid.shape

    for step in range(steps):
        ranges = line_scanning(rows, processors)

        args = [(grid, r[0], r[1], threshold) for r in ranges]

        with mp.Pool(processors) as pool:
            blocks = pool.map(update_block, args)

        grid = np.vstack(blocks)

        urban_ratio = np.sum(grid) / (rows * cols)
        print(f"Step {step + 1}/{steps} completed | Urban ratio: {urban_ratio:.3f}")

    return grid


# =========================
# MAIN FUNCTION
# =========================
if __name__ == "__main__":
    start_time = time.time()

    rows, cols, steps, processors, threshold = get_user_input()

    print("\nInitializing urban region...")
    grid = initialize_grid(rows, cols)

    print("\nRunning parallel urban CA simulation...\n")
    final_grid = parallel_urban_CA(grid, steps, processors, threshold)

    total_time = time.time() - start_time
    final_ratio = np.sum(final_grid) / (rows * cols)

    print("\n=== Simulation Complete ===")
    print(f"Final urbanization ratio: {final_ratio:.3f}")
    print(f"Total execution time: {total_time:.2f} seconds")



=== Parallel Urban Cellular Automata Simulation ===
Enter number of rows (e.g. 300): 300
Enter number of columns (e.g. 300): 250
Enter number of simulation steps (e.g. 20): 13
Enter number of processors (e.g. 4): 6
Enter urbanization threshold (0–1, e.g. 0.6): 0.6

Initializing urban region...

Running parallel urban CA simulation...

Step 1/13 completed | Urban ratio: 0.462
Step 2/13 completed | Urban ratio: 0.905
Step 3/13 completed | Urban ratio: 0.999
Step 4/13 completed | Urban ratio: 1.000
Step 5/13 completed | Urban ratio: 1.000
Step 6/13 completed | Urban ratio: 1.000
Step 7/13 completed | Urban ratio: 1.000
Step 8/13 completed | Urban ratio: 1.000
Step 9/13 completed | Urban ratio: 1.000
Step 10/13 completed | Urban ratio: 1.000
Step 11/13 completed | Urban ratio: 1.000
Step 12/13 completed | Urban ratio: 1.000
Step 13/13 completed | Urban ratio: 1.000

=== Simulation Complete ===
Final urbanization ratio: 1.000
Total execution time: 18.60 seconds
